In [ ]:
import torch
import torch.nn as nn
import numpy as np
from single_agent_env import SinglePlayerFootball, ACTION_SPACE_SIZE, STATE_SPACE_SIZE
from RL import DeepQNetworkAgent
from RL.utils import ReplayBuffer
torch.manual_seed(3407)
torch.cuda.manual_seed(3407)
np.random.seed(3407)

In [ ]:
class DQN(nn.Module):

    def __init__(self, input_shape, output_shape) -> None:
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_shape, 2048),
            nn.LeakyReLU(),
            nn.Linear(2048, 1024),
            nn.LeakyReLU(),
            nn.Linear(1024, 512),
            nn.LeakyReLU(),
            nn.Linear(512, 256),
            nn.LeakyReLU(),
            nn.Linear(256, 128),
            nn.LeakyReLU(),
            nn.Linear(128, output_shape),
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
agent = DeepQNetworkAgent(STATE_SPACE_SIZE, ACTION_SPACE_SIZE, device="cuda:1")
agent.create_model(DQN, lr=0.0003, y=0.99, e_decay=0.9999, batch=64, target_update_method="soft", tau=0.01, reward_norm_factor=300)
agent.create_buffer(ReplayBuffer(1_000_000, 10_000, STATE_SPACE_SIZE))

In [ ]:
train_id = "DQN_fixed_ball_normalized_medium_scratch_itr1"
env = SinglePlayerFootball(title=train_id)
best_score = 0
while env.running:
    state = env.reset(random_ball=False)
    while not env.loop_once():
        action = agent.policy(state)
        n_state, reward, done = env.step(action)
        agent.learn(state, action, n_state, reward, done)
        state = n_state
    if agent.episode_count % 100 == 0:
        avg = np.mean(agent.reward_history[len(agent.reward_history) - 100:])
        if avg > best_score:
            best_score = avg
            model_scripted = torch.jit.script(agent.model)
            model_scripted.save(f"best_models/{train_id}_{agent.episode_count}_{round(best_score, 6)}.pt")
del env

In [ ]:
with open(f'{train_id}_rewards.txt', 'w') as f:
    f.writelines([f"{round(item, 6)}\n" for item in agent.reward_history])

In [ ]:
agent.training = False
env = SinglePlayerFootball(title=train_id)
for _ in range(10):
    s = env.reset(ball_random=True)
    while not env.loop_once():
        s, _, _ = env.step(agent.policy(s))
del env